# Few-Shot Learning with Presto

## Notebook Overview 

1) Short introduction on Foundation Models and Presto
2) Definition of Few-Shot learning
3) Apply Presto to perfrom Few-Shot learning on a regression and a classification task

### 1) Foundation Models

A Foundation Model is a model trained on large and diverse unlabeled datasets to learn general patterns and features of the data. Thanks to its strong generalization capabilities, such a model can be adapted for a wide range of applications that use similar types of input data.

**Presto** (**P**retrained **Re**mote **S**ensing **T**ransf**o**rmer) is a foundation model trained on a large, unlabeled dataset of Sentinel-2, Sentinel-1, Meteorological and Topography pixel-timeseries data. It is able to capture long-range relationships across time and sensor dimensions, improving the signal-to-noise ratio and providing a concise, informative representation of the inputs. 
In this project, We made use of the [Presto](https://github.com/WorldCereal/prometheo.git) version developed in collaboration with WorldCereal

Originally trained on monthly composites, Presto has been refined to be able to ingest dekadal data and to be fine-tuned for regression and classification tasks.

### 2) Few-Shot Learning

Few-shot learning aims to develop models that can learn from a small number of labeled instances while enhancing generalization and performance on new, unseen examples.

Given a dataset with only a few annotated examples, we can fine-tune a pretrained foundation model to either directly handle the downstream task or generate compressed representations of the inputs (embeddings), which can then be used to train a machine learning model for the downstream task. We hereby show-case the former scenario, whose overview is depicted in the figure below.

<div style="text-align: center;">
    <img src="../images/ScaleAG_pipeline_overview_presto.jpg" alt="Overview of a Foundation Model fine tuned for different downstream tasks and applications." width="700" />
    <p><em>Overview of a Foundation Model fine tuned for different downstream tasks and applications.</em></p>
</div>

### 3) Implementing Few-Shot learning with Presto

In [33]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import geopandas as gpd
from scaleagdata_vito.presto.datasets_prometheo import ScaleAgDataset
from scaleagdata_vito.openeo.extract_sample_scaleag import generate_input_for_extractions, extract
from scaleagdata_vito.presto.utils import evaluate_finetuned_model
from scaleagdata_vito.presto.presto_df import load_dataset
from scaleagdata_vito.presto.utils import train_test_val_split, finetune_on_task, load_finetuned_model, get_pretrained_model_url, get_resources_dir
from scaleagdata_vito.presto.inference import PrestoPredictor, reshape_result, plot_results
from scaleagdata_vito.utils.map import ui_map
from scaleagdata_vito.utils.dateslider import date_slider
from scaleagdata_vito.openeo.extract_sample_scaleag import collect_inputs_for_inference

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Before we start...

**Check your data!** Investigate validity of geometries uniqueness of sample IDs, presence of outliers and so on before starting the extraction. Achieving good performance making use of a limited amount of data is a challening task per se. Therefore, **the quality of your data will greatly impact your final results.**

Data requirements:
- Points or Polygons (will be aggregated in points)
- Lat-Lon (crs:4326) 
- Format: parquet, GeoJSON, shapefile, GPKG
For each geometry:
- Date (if available) 
- Unique ID
- Annotations

Good practice:

Remove polygons close to borders (e.g. apply buffer) to ensure data are contained in the field
If the annotations are accurate, point geometries should be preferred. However, especially in regression tasks (i.e., continuous output values) such us yield estimation the target values might be noisy. In that case, we recommend subdividing the polygons in subfields of 20m x 20m (to cover more measurements) and computing the median yield for a smoother and more reliable target


#### Requirements for running the extractions
- Account in [Copernicus Data Space Ecosystem (CDSE)](https://dataspace.copernicus.eu/). You can sign up for free and have a monthly availability of 10000 credits.
- A dataset with valid geometries (Points or Polygons) in lat-lon projection.
- Preferably a dataset with unique IDs per sample 
- A labelled dataset. Not required for the extraction process, but for the following fine-tuning steps.

#### EO data extractions
In this first step, we extract for each sample in your dataset the required EO time series from CDSE using OpenEO.
For running the job, the user should indicate the following job_dictionary fields:

```python
    job_params = dict(
        output_folder=..., # where to save the extracted dataset
        input_df=..., # input georeferenced dataset to run the extractions for 
        start_date=..., # string indicating from which date to extract data  
        end_date=..., # string indicating until which date to extract the data 
        unique_id_column=..., # name of the column in the input_df containing the unique ID of the samples  
        composite_window=..., # "month" or "dekad" are supported. Default is "dekad"
    )
```
in particular:
- If the `date` information associated with the label is provided, the `start_date` of the time-series is automatically set to 9 months before the date, whereas the `end_date` is set to 9 months after. If `date` is not available, the user needs to manually indicate the desired `start_date` and `end_date` for the extractions.
- `composite_window` indicates the time-series granularity, which can be dekadal or monthly. 
  - `dekad`: each time step in the extracted time series corresponds to a mean-compositing operation on 10-days acquisitions. Accordingly with the start and end date, each month will be covered by 3 time steps which, by default, correspond to the 1st, 11th and 21th of the month. 
  - `month`: each time step in the extracted time series corresponds to a mean-compositing operation on 30-days acquisitions. Each month will be covered by 1 time step which, by default, correspond to the 1st of the month.

The following decadal/monthly time series will be extracted for the indicated time range:

- Sentinel-2 L2A data (all bands)
- Sentinel-1 VH and VV
- Average air temperature and precipitation sum derived from AgERA5
- Slope and elevation from Copernicus DEM

Presto accepts 1D time-series. Therefore, if Polygons are provided for the extractions, the latter are spatially aggregated in points which will correspond to the centroid lat lon geolocation.

### Regression task: potato yield estimation 

The data covers fields in Belgium during the growing season of 2022. Each field polygon was partitioned in subfields of 20m x 20m. The latter are partitioned into training, validation and test sets. 

**NOTE:** This is a very small dummy dataset with randomized yield values. No meaningful results are expected from using such data for training.

##### Data Extraction

In [54]:
output_folder = Path("/mnt/CEPH_PROJECTS/ScaleAgData/06_abhishek/few_shot/output_AGB/")
input_df = get_resources_dir() / "Fieldwork_2023_samples_uid.geojson"
start_date = "2023-04-01"
end_date = "2023-10-31"
unique_id_column = "uid"
composite_window = "dekad"

In [48]:
output_folder = Path("/mnt/CEPH_PROJECTS/ScaleAgData/06_abhishek/few_shot/output_v1/")
input_df1 = get_resources_dir() / "dummy_yield.geojson"
start_date = "2022-01-01"
end_date = "2022-12-31"
unique_id_column = "fieldname"
composite_window = "dekad"

In [55]:
# check input data structure 
gpd.read_file(input_df) 

,FOI/site,Farm,lat,lon,AGB_t/ha,date,parentfield,uid,geometry
0,R1-1,RNLI23,46.530865,11.431759,0.45524,2023-04-27,R1,1,POINT (11.43176 46.53086)
1,R1-2,RNLI23,46.530798,11.431515,1.05644,2023-04-27,R1,2,POINT (11.43152 46.5308)
2,R1-3,RNLI23,46.531055,11.431667,0.70364,2023-04-27,R1,3,POINT (11.43167 46.53106)
3,R5-1,RNLI23,46.530982,11.434240,0.56164,2023-04-27,R5,4,POINT (11.43424 46.53098)
4,R5-2,RNLI23,46.531125,11.434305,0.42204,2023-04-27,R5,5,POINT (11.4343 46.53112)
...,...,...,...,...,...,...,...,...,...
279,L1-2,LRKA23,46.453585,11.079331,0.30604,2023-10-19,L1,280,POINT (11.07933 46.45358)
280,L1-3,LRKA23,46.453720,11.079279,0.34044,2023-10-19,L1,281,POINT (11.07928 46.45372)
281,F1-1,FDCS23,46.433962,11.138077,0.21044,2023-10-19,F1,282,POINT (11.13808 46.43396)
282,F1-2,FDCS23,46.433860,11.138437,0.36684,2023-10-19,F1,283,POINT (11.13844 46.43386)


In [22]:
gpd.read_file(input_df1) 

,parentname,fieldname,median_yield,date,geometry
0,1574374,1574374-3,8259.734336,2022-09-21 18:54:15,"POLYGON ((3.18533 51.01671, 3.18533 51.01689, ..."
1,1422963,1422963-10,1030.794709,2022-10-25 10:53:03,"POLYGON ((5.11628 51.31606, 5.11629 51.31624, ..."
2,1676341,1676341-3,2266.873831,2022-10-12 14:44:10,"POLYGON ((5.03693 51.28708, 5.03693 51.28725, ..."
3,1650385,1650385-7,1438.154486,2022-09-30 14:21:27,"POLYGON ((2.85914 50.92523, 2.85914 50.92541, ..."
4,1500353,1500353-4,7375.198208,2022-10-15 10:38:17,"POLYGON ((5.59302 50.79991, 5.59303 50.80009, ..."
...,...,...,...,...,...
95,1462095,1462095-9,1630.024640,2022-09-21 17:15:03,"POLYGON ((5.00341 50.83186, 5.00342 50.83204, ..."
96,1571140,1571140-4,9685.894003,2022-10-03 18:42:00,"POLYGON ((5.16576 51.30617, 5.16576 51.30635, ..."
97,1369340,1369340-2,8770.054389,2022-10-18 06:50:47,"POLYGON ((3.37756 51.02309, 3.37756 51.02327, ..."
98,1671783,1671783-2,8306.438809,2022-09-20 18:30:19,"POLYGON ((5.07273 50.81302, 5.07274 50.8132, 5..."


In [56]:
job_params = dict(
    output_folder=output_folder,
    input_df=input_df,
    unique_id_column=unique_id_column,
    composite_window=composite_window,
)
extract(generate_input_for_extractions(job_params))

2025-12-17 13:47:17,129|extraction_pipeline|INFO:  Loading input dataframe from /mnt/CEPH_PROJECTS/ScaleAgData/06_abhishek/few_shot/scaleag-vito-v1/resources/Fieldwork_2023_samples_uid.geojson.
2025-12-17 13:47:17,141|extraction_pipeline|INFO:  Preparing the job dataframe.
2025-12-17 13:47:17,142|extraction_pipeline|INFO:  Performing splitting by s2 grid...
/home/absingh/miniconda3/lib/python3.9/site-packages/openeo_gfmap/manager/job_splitters.py:113: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  polygons["centroid"] = polygons.geometry.centroid
2025-12-17 13:47:17,442|extraction_pipeline|INFO:  Dataframes split to jobs, creating the job dataframe...
  0%|                                                                                                                        | 0/6 [00:00<?, ?it/s]/mnt/CEPH_PROJECTS/ScaleAgData/06_abhishek/few_shot/s

Authenticated using refresh token.


2025-12-17 13:56:59,541|openeo_gfmap.manager|INFO:  Parsed item timeseries.parquet from job j-2512171247204ea399b165eea6c8aaa3
2025-12-17 13:56:59,591|openeo_gfmap.manager|INFO:  Adding 1 items to the STAC collection...
2025-12-17 13:56:59,591|openeo_gfmap.manager|INFO:  Job j-2512171247204ea399b165eea6c8aaa3 and post job action finished successfully.
2025-12-17 13:58:20,791|openeo_gfmap.manager|INFO:  Parsed item timeseries.parquet from job j-2512171247374abbaf365800f3822b1f
2025-12-17 13:58:20,839|openeo_gfmap.manager|INFO:  Adding 1 items to the STAC collection...
2025-12-17 13:58:20,839|openeo_gfmap.manager|INFO:  Job j-2512171247374abbaf365800f3822b1f and post job action finished successfully.
2025-12-17 14:06:47,961|openeo_gfmap.manager|INFO:  Parsed item timeseries.parquet from job j-251217125701462cb0b5320be2a231f4
2025-12-17 14:06:48,011|openeo_gfmap.manager|INFO:  Adding 1 items to the STAC collection...
2025-12-17 14:06:48,012|openeo_gfmap.manager|INFO:  Job j-25121712570146

Once the dataset will be extracted, it can be loaded with the `load_dataset` function by specifying the path where the `.parquet` files have been downloaded. Moreover, the following manipulations of the dataset are also possible:

- `window_of_interest`: the user can specify a time window of interest out of the whole available time-series. `start_date` and `end_date` should be provided as strings in a list.
- `use_valid_time`: the user might want to define the window of interest based on the `date` the label is associated with. If so, also `required_min_timesteps` should be provided
- `buffer_window`: buffers the `start_date` and `end_date` by the number of time steps here specified  

In the following cell, we load the extracted dataset for 1 year of data.

**NOTE:** this code currently assumes that we are dealing with 1 year of data falling in the same time period

##### Presto datasets initialization

In [57]:
df = load_dataset(
    files_root_dir=output_folder,
    window_of_interest=[start_date, end_date],
    composite_window=composite_window,
)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 23.49it/s]


The following step splits the data into train, test and val datasets. the split can be performed by uniform sampling or by group sampling. The former is usually more suitable for the binary and multiclass classification tasks, to ensure the data distribution is represented in all the 3 sets. The latter is more specific for cases where we want to avoid data autocorrelation and so data leakage between training and val/test sets.
In the case of yield estimation, for instance, we often have samples coming from the same field. So we might want to separate the data based on the field they belong to to better test the model generalization capabilities.
Therefore:
- `uniform_sample_by`: pass the name of the column in the dataframe to perform uniform sampling on
- `group_sample_by`: pass the name of the column in the dataframe to perform the group sampling on

`sampling_fraction` indicates the proportion of the training set out of the whole dataset. the remaining percentage will be equally devided into validation and test sets.

In [58]:
#df_train, df_val, df_test = train_test_val_split(df=df, group_sample_by="parentname", sampling_frac=0.8)
df_train, df_val, df_test = train_test_val_split(df=df, group_sample_by="uid", sampling_frac=0.8)

2025-12-17 14:33:14.364 | INFO     | scaleagdata_vito.presto.utils:train_test_val_split:565 - Training set size: 227
2025-12-17 14:33:14.365 | INFO     | scaleagdata_vito.presto.utils:train_test_val_split:566 - Validation set size: 28
2025-12-17 14:33:14.365 | INFO     | scaleagdata_vito.presto.utils:train_test_val_split:567 - Test set size: 29


We now set up the parameters needed for initializing presto datasets for the specific task:
- `num_timesteps`: can be inferred by the max number of the `available_timesteps` 
- `target_name`: name of the column containing the target data
- `upper_bound` and `lower_bound`: these should be set to the min and max of the distribution. Therefore, it is important to get rid of potential outlaiers beforehand.

**NOTE:** upper and lower bounds are also used to normalize the targets during the training process. Therefore it is important to keep track of such values to convert the predictions to the original units in the inference step!  

In [59]:
# visualize distribution to check for outliers to exclude if needed
num_timesteps = df.available_timesteps.max()
task_type = "regression"
target_name = "AGB_t/ha"
upper_bound = df[target_name].max() 
lower_bound = 0 

We Initialize the training, validation and test datasets objects to be used for training Presto.

In [60]:
# initialize datasets
train_ds = ScaleAgDataset(
    df_train,
    num_timesteps=num_timesteps,
    task_type=task_type,
    target_name=target_name,
    composite_window=composite_window,
    upper_bound=upper_bound,
    lower_bound=lower_bound,
)
val_ds = ScaleAgDataset(
    df_val,
    num_timesteps=num_timesteps,
    task_type=task_type,
    target_name=target_name,
    composite_window=composite_window,
    upper_bound=upper_bound,
    lower_bound=lower_bound,
)
test_ds = ScaleAgDataset(
    df_test,
    num_timesteps=num_timesteps,
    task_type=task_type,
    target_name=target_name,
    composite_window=composite_window,
    upper_bound=upper_bound,
    lower_bound=lower_bound,
)

2025-12-17 14:34:07.042 | INFO     | scaleagdata_vito.presto.datasets_prometheo:set_num_outputs:140 - Setting number of outputs to 1 for regression task.
2025-12-17 14:34:07.044 | INFO     | scaleagdata_vito.presto.datasets_prometheo:set_num_outputs:140 - Setting number of outputs to 1 for regression task.
2025-12-17 14:34:07.046 | INFO     | scaleagdata_vito.presto.datasets_prometheo:set_num_outputs:140 - Setting number of outputs to 1 for regression task.


##### Presto Finetuning

In this section Presto will be Fine-Tuned in a supervised way for the target downstream task. first we set up the following experiment parameters:

- `output_dir` : where to dave the model 
- `experiment_name` : the model name
- `pretrained_model_path` : pretrained presto model to start the fine tuning from. Can be a string indicating the path to the model or a url 

In [63]:
# set models hyperparameters
model_output_dir = Path("/mnt/CEPH_PROJECTS/ScaleAgData/06_abhishek/few_shot/output_AGB/")
experiment_name = "presto-agb-dekv1"

In [64]:
# Construct the model with finetuning head starting from the pretrained model
finetuned_model = finetune_on_task(
    train_ds=train_ds,
    val_ds=val_ds,
    pretrained_model_path=get_pretrained_model_url(composite_window=composite_window),
    output_dir=model_output_dir, 
    experiment_name=experiment_name,
    num_workers=0,
    )
evaluate_finetuned_model(finetuned_model, test_ds, num_workers=0, batch_size=32)

2025-12-17 14:55:23.964 | INFO     | scaleagdata_vito.presto.utils:finetune_on_task:460 - Finetuning the model on regression task
2025-12-17 14:55:23.966 | INFO     | prometheo.finetune:_setup:223 - Using output dir: /mnt/CEPH_PROJECTS/ScaleAgData/06_abhishek/few_shot/output_AGB


Finetuning:   0%|          | 0/50 [00:00<?, ?it/s]

Training:   0%|          | 0/227 [00:00<?, ?it/s]

2025-12-17 14:55:24.385 | INFO     | prometheo.finetune:_train_loop:178 - PROGRESS after Epoch 1/50: Epoch 1/50 | Train Loss: 0.0022 | Val Loss: 0.1403 | Best Loss: 0.1403 (improved)


Training:   0%|          | 0/227 [00:00<?, ?it/s]

...

Training:   0%|          | 0/227 [00:00<?, ?it/s]

2025-12-17 14:55:27.143 | INFO     | prometheo.finetune:_train_loop:178 - PROGRESS after Epoch 8/50: Epoch 8/50 | Train Loss: 0.0004 | Val Loss: 0.0358 | Best Loss: 0.0317 (no improvement for 2 epochs)


Training:   0%|          | 0/227 [00:00<?, ?it/s]

2025-12-17 14:55:27.524 | INFO     | prometheo.finetune:_train_loop:161 - Early stopping!
2025-12-17 14:55:27.543 | INFO     | prometheo.finetune:run_finetuning:327 - Finetuning done
2025-12-17 14:55:27.544 | INFO     | scaleagdata_vito.presto.utils:evaluate_finetuned_model:115 - Evaluating the finetuned model on regression task


{'RMSE': 2.546637120283533,
 'R2_score': -0.2759532928466797,
 'explained_var_score': -0.27283310890197754,
 'MAPE': 5.678121566772461}

##### Inference using Fine-Tuned end-to-end Presto

In this section, we apply the fine tuned model to generate a yield map on an unseen area. 
We need to indicate the spatial and temporal extent. The 2 cells below, offer a simple way for the user to provide these information and perform once again the extraction from CDSE of the EO time-series required by Presto. 
We also need to indicate the `output_dir` of where to save the datacube of the extraction, its `output_filename` and the `composite_window` which will be the same as used for finetuning the model.

In [ ]:
map = ui_map(area_limit=7)

In [ ]:
# select 1 year of data
slider = date_slider()

In [ ]:
output_dir = Path("/home/giorgia/Private/data/scaleag/demo/regression")
output_filename = "inference_area"
inference_file = output_dir / f"{output_filename}.nc"

In [ ]:
collect_inputs_for_inference(
    spatial_extent=map.get_extent(),
    temporal_extent=slider.get_processing_period(),
    output_path=output_dir,
    output_filename=f"{output_filename}.nc",
    composite_window=composite_window,
)

Once the datacube has been extracted, we can perform the inference task using the finetuned model and visualize the predicted map. 

In [ ]:
inference_file = get_resources_dir() / "inference_area_tevuren.nc"
mask_path = get_resources_dir() / "LPIS_flanders_potatoes_2022.tif"

In [ ]:
finetuned_model = load_finetuned_model(model_output_dir / experiment_name, task_type=task_type)
presto_model = PrestoPredictor(
    model=finetuned_model,
    batch_size=50,
    task_type=task_type,
    composite_window=composite_window,
)

predictions = presto_model.predict(inference_file, upper_bound=upper_bound, lower_bound=lower_bound, mask_path=mask_path)
predictions_map = reshape_result(predictions, path_to_input_file=inference_file)

In [ ]:
plot_results(prob_map=predictions_map, path_to_input_file=inference_file, task=task_type, ts_index=5)

### Binary task: crop/no-crop

Now we test the few-shot learning on a binary task. We Fine-Tune presto on datapoints sampled from Flanders on 2021. This time, the dataset is the result of a monthly compositing. We initialize the parameters for the dataset preparation accordingly  

In [ ]:
output_folder = Path("/home/giorgia/Private/data/scaleag/demo/worldcereal/")
input_df = get_resources_dir() / "dummy_cropland.geojson"
task_type = "binary"
target_name = "LANDCOVER_LABEL"
composite_window = "month"
unique_id_column = "sample_id"

In [ ]:
job_params = dict(
    output_folder=output_folder,
    input_df=input_df,
    unique_id_column=unique_id_column,
    composite_window=composite_window,
)
extract(generate_input_for_extractions(job_params))

##### Presto datasets initialization

In [ ]:
df = load_dataset(
    files_root_dir=output_folder,
    composite_window=composite_window,
)
train_df, val_df, test_df = train_test_val_split(df=df, uniform_sample_by=target_name, sampling_frac=0.6)

In the context of a binary classification task, specifying the upper and lower bounds won't be necessary anymore. In case we are dealing with multiclass labels, we can convert the problem into a binary classification task by providing the `positive_labels` argument to the `ScaleAgDatset` class. The list of labels passed as value to `positive_labels` indicates which subset of classes should be interpreted as positive class (here "crop"). All the other labels will therefore be interpreted as negative class (here "no-crop").

In [ ]:
num_timesteps = df.available_timesteps.max()
positive_labels = [10, 11, 12, 13]

In [ ]:
train_ds = ScaleAgDataset(
    train_df,
    num_timesteps=num_timesteps,
    task_type=task_type,
    target_name=target_name,
    positive_labels=positive_labels,
    composite_window=composite_window,
)

val_ds = ScaleAgDataset(
    val_df,
    num_timesteps=num_timesteps,
    task_type=task_type,
    target_name=target_name,
    positive_labels=positive_labels,
    composite_window=composite_window,
)

test_ds = ScaleAgDataset(
    test_df,
    num_timesteps=num_timesteps,
    task_type=task_type,
    target_name=target_name,
    positive_labels=positive_labels,
    composite_window=composite_window,
)

##### Presto Finetuning

In this section Presto will be Fine-Tuned in a supervised way for the target downstream task. Once again, we set up the experiment parameters.

In [ ]:
experiment_name = "presto_wc_ft_crop"
model_output_dir = Path("/home/giorgia/Private/data/scaleag/demo/worldcereal/")

In [ ]:
finetuned_model = finetune_on_task(
                train_ds=train_ds,
                val_ds=val_ds,
                pretrained_model_path=get_pretrained_model_url(composite_window=composite_window),
                output_dir=model_output_dir, 
                experiment_name=experiment_name,
                num_workers=0,
            )
evaluate_finetuned_model(finetuned_model, test_ds, num_workers=0, batch_size=32)

##### Inference using Fine-Tuned end-to-end Presto

We now apply the finetuned model to an unseen area to perform the classification task.

In [ ]:
finetuned_model = load_finetuned_model(model_output_dir / experiment_name, task_type=task_type)
inference_file = get_resources_dir() / "worldcereal_preprocessed_inputs.nc"

In [ ]:
presto_model = PrestoPredictor(
    model=finetuned_model,
    batch_size=50,
    task_type=task_type,
    composite_window=composite_window,
)

predictions = presto_model.predict(inference_file)
prob_map = reshape_result(predictions, path_to_input_file=inference_file)
pred_map = presto_model.get_predictions(prob_map, threshold=0.75)
plot_results(prob_map=prob_map, pred_map=pred_map, path_to_input_file=inference_file, task=task_type, ts_index=4)